# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

3. Визуализируйте граф вычислений для задачи 2.

## Лабораторная работа 11

In [1]:
import dask.array as da
import dask.dataframe
import h5py
import numpy as np

1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [2]:
with h5py.File("C:/Users/Александр/Downloads/TOBD/11_dask_array/11_dask_array_data/minutes_n_ingredients_full.hdf5", "r") as hdf:
    resource = hdf.get('recipe')
    resource = np.asarray(resource)
    resource = da.asarray(resource, chunks=(100_000, 3)) # количество используемых строк на раздел 
resource

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [3]:
resource.compute() # запуск фактических вычислений 

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      60,       2],
       [ 713836,       0,       9],
       [ 660699,      64,       8]], dtype=int64)

In [4]:
%%time
resource[:, 1].mean().compute()

Wall time: 17 ms


1004.2080517575215

In [5]:
%%time
resource[:, 2].mean().compute()

Wall time: 16.2 ms


5.419800800936711

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [6]:
with h5py.File("C:/Users/Александр/Downloads/TOBD/11_dask_array/11_dask_array_data/minutes_n_ingredients_full.hdf5", "r") as hdf:
    resource = hdf.get('recipe')
    resource = np.asarray(resource)
    resource = da.asarray(resource, chunks=(10_000, 3)) # количество используемых строк на раздел 
resource

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(10000, 3), chunktype=numpy.ndarray>

In [7]:
%%time
resource[:, 1].mean().compute()

Wall time: 93.5 ms


1004.2080517575215

In [8]:
%%time
resource[:, 2].mean().compute()

Wall time: 81.6 ms


5.419800800936711

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [9]:
median = da.median(resource[:, 1], axis=0).compute()

In [10]:
median

32.0

In [11]:
resource[resource[:, 1] < median].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [12]:
unique_value, key_counts = np.unique(resource[:, 2], return_counts=True)
dict(zip(unique_value.compute(), key_counts.compute()))

{1: 222071,
 2: 224158,
 3: 229388,
 4: 234948,
 5: 240720,
 6: 244360,
 7: 247181,
 8: 246747,
 9: 246816,
 10: 22430,
 11: 19094,
 12: 15165,
 13: 11640,
 14: 8284,
 15: 6014,
 16: 4145,
 17: 2793,
 18: 1913,
 19: 1279,
 20: 852,
 21: 529,
 22: 346,
 23: 244,
 24: 178,
 25: 107,
 26: 68,
 27: 55,
 28: 33,
 29: 22,
 30: 20,
 31: 13,
 32: 5,
 33: 4,
 34: 3,
 35: 4,
 36: 1,
 37: 2,
 38: 1,
 39: 1,
 40: 2,
 43: 1}

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [13]:
dask_df = dask.dataframe.from_array(resource)
dask_df

,0,1,2
npartitions=224,,,
0,int64,int64,int64
10000,...,...,...
...,...,...,...
2230000,...,...,...
2231636,...,...,...


In [14]:
quantile = dask_df[1].quantile(0.75)

In [15]:
dask_df[dask_df[1] < quantile][1].max().compute()

49

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [16]:
my_recipe = da.array([2323, 23])

resource[(da.power(resource[:, [1, 2]] - my_recipe, 2)).mean(axis=1).argmin()].compute()

array([178596,   2340,     13], dtype=int64)

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент